# Sentinel 1 and 2, and PALSAR input data
#### Description of content here:

## Intro Section

In [1]:
import os
import ee
import folium

In [2]:
ee.Initialize()

In [3]:
import geemap
from geemap import ml
import pandas as pd
from sklearn import ensemble

In [4]:
geemap.ee_initialize()

In [5]:
import ipywidgets as widgets

In [6]:
from ipynb.fs.full.indices1 import *
# from ipynb.fs.full.mapclient import *

In [7]:
iDate = '2015-01-01'
fDate = '2022-01-01'

In [8]:
start = ee.Date.fromYMD(2021,1,1)
end = ee.Date.fromYMD(2022,1,1)

In [9]:
# Sentinel-1 half-year medium composites:
#
start1_S1 = ee.Date.fromYMD(2021,1,1);
end1_S1 = ee.Date.fromYMD(2021,6,30);

start2_S1 = ee.Date.fromYMD(2021,7,1);
end2_S1 = ee.Date.fromYMD(2021,12,31);


In [10]:
nw_2box = ee.FeatureCollection('projects/ee-pierdubo/assets/nw_2box')
nw1 = (nw_2box
       .filter(ee.Filter.eq('sel', 1)))

In [11]:
# geom = ee.Geometry.Polygon(nw1)
geom = nw1.geometry()

In [12]:
dTrain = ee.FeatureCollection('projects/ee-pierdubo/assets/dTrain_44bds_vJan27')

In [ ]:
# cst = ee.FeatureCollection('users/yanchojo/gadm36_mmr_0_shore')

In [13]:
topo = ee.ImageCollection('users/yanchojo/topo')

In [ ]:
# pp.pprint(dTrain.limit(10).getInfo())

In [14]:
Mangr = ee.FeatureCollection("projects/ee-pierdubo/assets/gmw2016_sel1")
openW = ee.FeatureCollection('projects/ee-pierdubo/assets/OW_12months_sample_north_sel2')
lc7 = ee.FeatureCollection('projects/ee-pierdubo/assets/lc_7class_v02')

In [ ]:
nfc = Mangr.merge(openW).merge(lc7)

In [ ]:
print(nfc.size().getInfo())

In [ ]:
# geom

In [15]:
import pprint
pp = pprint.PrettyPrinter(depth=6, compact=True)

In [ ]:
# pprint.pprint(nfc.limit(10).getInfo())

In [26]:
# AOI for NWestern area, using a polyline
aoi = ee.Geometry.LineString(
    [[49.09178474992574, -11.924273142945154],
     [48.66331795305074, -13.275179628809104],
     [47.818428215464124, -14.41247011042455],
     [48.179964869881715, -14.846057452723358],
     [48.08998151060017, -15.275048458710872],
     [47.753046040851736, -14.483073303752759],
     [46.60887459367574, -15.668473695347696],
     [44.40062264055074, -16.196686141581498]])

In [27]:
geom = aoi

In [28]:
geom = ee.Geometry.Rectangle([[48.815, -13.233], [48.932, -13.309]])

In [53]:
# Set visualization parameters for MVI
mvi_vis_params = {
    'min': -3, 'max': 5,
    'palette': ['04fffe','CE7E45','DF923D','F1B555','FCD163','99B718','74A901','66A000',
                '529400','3E8601','207401','056201','004C00','023B01','012E01','011D01','011301']    
}

## Main Functions: Cloud Masking, doIndices, PrepareS2, etc.

In [16]:
# Function to cloud mask from the QA60 (Sentinel-2) and from the pixel_qa band of Landsat 8 SR data
def maskS2clouds(image):
    QA60 = image.select(['QA60'])
    clouds = QA60.bitwiseAnd(1 << 10).Or(QA60.bitwiseAnd(1 << 11))
    return image.updateMask(clouds.Not())

In [35]:
# Calculate spectral indices for all bands in collection
# @param {ee.ImageCollection} collection Landsat image collection
# @returns {ee.ImageCollection} Landsat image with spectral indices
#
def doIndS2(image):
    def func_gev(image):
        NDVI = calcNDVI(image)
        gNDVI = addGNDVI(image)
        NBR = calcNBR(image)
        EVI = calcEVI(image)
        EVI2 = calcEVI2(image)
        NDMI = addNDMI(image)
        NDWI = addNDWI(image)
        MNDWI = addMNDWI(image)
        MVI = addMVI(image)
        CMRI = addCMRI(image)
        # MMRI = addMMRI(image)
        EBBI = addEBBI(image)
        ARI2 = addARI2(image)
        ChlR = addChlR(image)
        BSI = addBSI(image)
        IBI = addIBI(image)
        SAVI = addSAVI(image)        
        # TC = tcTrans(image)
        # NDFI function requires surface reflectance bands only
        # BANDS = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']
        # NDFI = calcNDFI(image.select(BANDS))
        return image.addBands([NDVI, gNDVI, NBR, EVI, EVI2, NDMI, NDWI, MNDWI, ARI2, ChlR, MVI, CMRI, SAVI, BSI, IBI, EBBI]) #
    return image.map(func_gev)

In [18]:
# Prepare Sentinel-2 
#
def prepareS2(image):
    bandList = ['B2', 'B3', 'B4', 'B5','B6','B7', 'B8', 'B8A','B11', 'B12', 'QA60']
    nameList = ['BLUE','GREEN','RED', 'Redg1', 'Redg2', 'Redg3','NIR','NIRa','SWIR1','SWIR2','QA60']
    scaling = 10000
    # scaling = 1
    scaled = ee.Image(image).select(bandList).rename(nameList).divide(ee.Image.constant(scaling))
    return ee.Image(image).addBands(scaled)

## Apply Functions to Collect Satellite Images, Sentinel-2 and Sentinel-1

In [36]:
# Generate filtered (date and cloud masks) collection of Sentinel-2 images
 # @param {ee.Image} geom Geometry used to filter the collection
 # @param {String} startDate Initial date to filter the collection
 # @param {String} endDate Final date to filter the collection
 # @param {number} cpp stands for 'Cloudy Pixel Percentage'
 # @returns {ee.ImageCollection} Filtered Sentinel-2 collection
#
def genCollS2(aoi, iDate, fDate, cpp):
    collect_S2 = (ee.ImageCollection('COPERNICUS/S2')
                  .filterDate(iDate, fDate)
                  .filterBounds(geom)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cpp))
                  .map(maskS2clouds))
    prep_S2 = collect_S2.map(prepareS2)
    indices = doIndS2(prep_S2).select(['BLUE','GREEN','RED','NIR', 'NIRa','SWIR1','SWIR2',
                                       'Redg1', 'Redg2', 'Redg3','NDVI','gNDVI','NBR','EVI','EVI2',
                                       'NDMI','NDWI','MNDWI','ARI2','ChlR','MVI','CMRI','SAVI','BSI','IBI','EBBI'])  # 'MMRI',
    return ee.ImageCollection(indices)    

In [ ]:
# Extra codes that were part of 'genCollS2' JS version
# def func_ren(image):
    #     img = image.select(['B2','B3','B4','B8','B11','B12','QA60'])
    #     .rename(['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','QA60']).divide(10000)
    #     return img
    # .map(func_ren)
    # .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 75)
    # return ee.ImageCollection(indices) #collect_S2; #.select('B2','B3','B4','B8','B11','B12') #ee.ImageCollection(

In [30]:
def genCollS1(aoi, iDate, fDate, orbit, smoothing_radius, kernelType):
    img = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterDate(iDate, fDate)
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
           .filter(ee.Filter.eq('orbitProperties_pass', orbit))
           .filterMetadata('resolution_meters', 'equals', 10)
           .filterBounds(geom)
           .select(['V.','angle']))
    
    def fun_a(img):
        geom = img.geometry()
        angle = img.select('angle')
        edge = img.select('VV').lt(-30.0)
        maskedImg = img.mask().And(edge.Not())
        return img.updateMask(maskedImg)
    
    def fun_b(img):
        fmean = img.select('V.').focal_mean(smoothing_radius, kernelType, 'meters')
        ratio = fmean.select('VH').subtract(fmean.select('VV')).rename('VHrVV')
        sigma0Pow = ee.Image.constant(10).pow(fmean.divide(10.0))
        ndsar = sigma0Pow.normalizedDifference(['VH', 'VV']).float().rename('NDS1')
        return img.select().addBands(fmean).addBands(ratio).addBands(ndsar)
    
    return img.map(fun_b).map(fun_a)

In [ ]:
def getS2(geom, iDate, fDate, cpp):
    s2 = ee.ImageCollection('COPERNICUS/S2')
    s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')
    
    start = ee.Date(iDate)
    end = ee.Date(fDate)
    s2 = (s2.filterDate(start, end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cpp))
          .select(['B7', 'B8', 'B8A', 'B10']))
    s2c = s2c.filterDate(start, end)
    s2Sr= (s2Sr.filterDate(start, end)
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cpp))
           .select(['B2', 'B3', 'B4', 'B5','B6','B7','B8','B8A','B11','B12']))
    if (geom):
        s2 = s2.filterBounds(geom)
        s2c = s2c.filterBounds(geom)
        s2Sr = s2Sr.filterBounds(geom)
    def indexJoin(collectionA, collectionB, propertyName):
        joined = (ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(**{
            'primary': collectionA,
            'secondary': collectionB,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index',
            })
        })))
        def fun_zyp(image):
            return image.addBands(ee.Image(image.get(propertyName)))
        return joined.map(fun_zyp)
     
    def maskImage(image):
        cdi = ee.Algorithms.Sentinel2.CDI(image)
        s2c = image.select('probability')
        cirrus = image.select('B10').multiply(0.0001)
        
        isCloud = s2c.gt(65).And(cdi.lt(-0.5)).Or(cirrus.gt(0.01))
        
        isCloud = isCloud.focal_min(3).focal_max(16)
        isCloud = isCloud.reproject(**{'crs': cdi.projection(), 'scale': 20})
        
        shadowAzimuth = (ee.Number(90)
                         .subtract(ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE'))))
        
        isCloud = isCloud.directionalDistanceTransform(shadowAzimuth, 10)
        isCloud = isCloud.reproject(**{'crs': cdi.projection(), 'scale': 100})
        
        isCloud = isCloud.select('distance').mask()
        return (image.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'])
         .rename(['BLUE','GREEN','RED','Redg1', 'Redg2', 'Redg3','NIR','NIRa','SWIR1','SWIR2'])
         .divide(10000).updateMask(isCloud.Not())
         .set('system:time_start', ee.Image(image.get('l1c')).get('system:time_start')))
         
    withCloudProbability = indexJoin(s2Sr, s2c, 'cloud_probability')
    withS2L1C = indexJoin(withCloudProbability, s2, 'l1c')
    
    coll = ee.ImageCollection(withS2L1C.map(maskImage))
    indices = doIndS2(coll).select(['BLUE','GREEN','RED','NIR', 'NIRa','SWIR1','SWIR2',
                                       'Redg1', 'Redg2', 'Redg3','NDVI','gNDVI','NBR','EVI','EVI2',
                                       'NDMI','NDWI','MNDWI','ARI2','ChlR','MVI','CMRI','MMRI','SAVI','BSI','IBI','EBBI']) 
    return ee.ImageCollection(indices) 

In [ ]:
s2Get = getS2(aoi, start, end, 25)

### Collect Sentinel-1 

In [31]:
s1Coll = genCollS1(aoi, iDate, fDate, 'DESCENDING', 30, 'circle')

In [32]:
SARV_1 = (s1Coll.select(['VH','VV','VHrVV','NDS1'])
          .filterDate(start1_S1, end1_S1).median()
          .rename('VH_1', 'VV_1', 'VHrVV_1', 'NDS1_1'))

In [33]:
SARV_2 = (s1Coll.select(['VH','VV','VHrVV','NDS1'])
          .filterDate(start2_S1, end2_S1).median()
          .rename('VH_2', 'VV_2', 'VHrVV_2', 'NDS1_2'))

### Collect Sentinel-2

In [37]:
s2Coll = genCollS2(aoi, start, end, 25)

In [38]:
s2Coll.median().bandNames().getInfo()

['BLUE',
 'GREEN',
 'RED',
 'NIR',
 'NIRa',
 'SWIR1',
 'SWIR2',
 'Redg1',
 'Redg2',
 'Redg3',
 'NDVI',
 'gNDVI',
 'NBR',
 'EVI',
 'EVI2',
 'NDMI',
 'NDWI',
 'MNDWI',
 'ARI2',
 'ChlR',
 'MVI',
 'CMRI',
 'SAVI',
 'BSI',
 'IBI',
 'EBBI']

In [ ]:
# An array of 'dates' from an ee Image Collection
def func_dateProp(img):
    return ee.Feature(None, {'date': img.date().format('YYYY-MM-dd')})

In [ ]:
### JavaScript version:
# var dates = S1coll
#     .map(function(image) {
#       return ee.Feature(null, {'date': image.date().format('YYYY-MM-dd')});
#     })
#     .aggregate_array('date');

In [ ]:
dateS2 = s2Coll.map(func_dateProp).aggregate_array('date')

In [ ]:
dateS2.getInfo()

In [ ]:
s1Coll_f = s1Coll.first()

In [ ]:
s2Coll_f = s2Coll.first()

In [ ]:
ee.Date(s2Coll_f.get('system:time_start')).format('YYYY-MM-dd').getInfo()

In [ ]:
s2Coll_fProps = geemap.image_props(s2Coll_f)

In [ ]:
s1Coll_fStats = geemap.image_stats(s1Coll_f, scale=30)
s1Coll_fStats.getInfo()

In [ ]:
s2Coll_fProps.getInfo()

In [ ]:
def getCloudScores(img):
    value = ee.Image(img).get('CLOUDY_PIXEL_PERCENTAGE')
    return ee.Feature(None, {'score': value})

In [ ]:
cloudS2 = s2Coll.map(getCloudScores).aggregate_array('score')

In [ ]:
cloudS2.getInfo()

In [ ]:
## Just testing a simplified version for S2 image coll
#
coll_S2 = (ee.ImageCollection('COPERNICUS/S2')
              .filterDate(iDate, fDate)
              .filterBounds(geom)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))
              .map(maskS2clouds)
              .map(prepareS2).select(['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','QA60']))


### Display Results for S1 and S2

In [39]:
image = s2Coll.median()

In [40]:
image.bandNames().getInfo()

['BLUE',
 'GREEN',
 'RED',
 'NIR',
 'NIRa',
 'SWIR1',
 'SWIR2',
 'Redg1',
 'Redg2',
 'Redg3',
 'NDVI',
 'gNDVI',
 'NBR',
 'EVI',
 'EVI2',
 'NDMI',
 'NDWI',
 'MNDWI',
 'ARI2',
 'ChlR',
 'MVI',
 'CMRI',
 'SAVI',
 'BSI',
 'IBI',
 'EBBI']

In [41]:
# ndviSD = s2Coll.map(addNDVIsd).median() # Returned '0' Hmmm!!
ndviSD = s2Coll.select('NDVI').reduce(ee.Reducer.stdDev()).rename('ndviSD')

In [42]:
image = image.addBands(ndviSD)

In [43]:
image = image.addBands(SARV_1)
image = image.addBands(SARV_2)

In [ ]:
geom1 = ee.Geometry.Rectangle([[48.1227, -13.2193], [48.9772, -13.7135]])

In [ ]:
Map.addLayer(nw_2box, {'color': 'blue'}, 'nfc')

In [ ]:
Map.addLayer(nfc+, {'color': 'yellow'}, 'nfc+')

In [ ]:
chlr2 = s2Coll.map(addChlR).median()
image = image.addBands(chlr2)

In [ ]:
Map.addLayer(mmrix, mvi_vis_params , 'mmrix')

In [45]:
Map.addLayer(coll_S2.select(['SWIR1','NIR','GREEN']).median(), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Sentinel-2 Coll')

NameError: name 'coll_S2' is not defined

In [ ]:
Map.addLayer(s2Coll.median(), {}, 'S2 Collection_all')

In [46]:
Map.addLayer(s2Coll.select(['SWIR1','NIR','GREEN']).median(), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Sentinel-2 Coll')

In [55]:
Map.addLayer(SARV_1.select(['VV_1','VH_1','VHrVV_1']), {'bands': ['VV_1','VH_1','VHrVV_1'], 'min':-16, 'max':-5, 'gamma': 0.90}, 'S1 Jan-Jul')
Map.addLayer(SARV_2.select(['VV_2','VH_2','VHrVV_2']), {'bands': ['VV_2','VH_2','VHrVV_2'], 'min':-16, 'max':-5, 'gamma': 0.90}, 'S1 Jul-Dec')

In [54]:
Map.addLayer(image.select('MVI').clip(nw1), mvi_vis_params, 'Image')

In [ ]:
Map.addLayer(img_rfClass.randomVisualizer(), {}, 'classified')

In [ ]:
Map.addLayer(s2Get.select(['SWIR1','NIR','GREEN']).median().clip(geom1), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'S2 get', 1, 0.8)

In [44]:
Map = geemap.Map(center=(-13.5, 48.4), zoom=8)
Map

Map(center=[-13.5, 48.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [ ]:
mmrix = s2Coll.map(addMMRIx).median()

In [ ]:
mmrix.bandNames().getInfo()

In [ ]:
# 17. Calculate/addBands: MMRI Modular Mangrove Recognition Index (MNDWI-NDVI)(MNDWI+NDVI); Diniz et al. 2018
#
def addMMRIx(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR', 'RED']).abs()
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).abs()
    mmri = ee.Image(mndwi.subtract(ndvi)).divide(ee.Image(mndwi.add(ndvi))).rename(['MMRIx'])
    return mmri    

In [ ]:
chlr2 = s2Coll.map(addChlR).median()
image = image.addBands(chlr2)

In [ ]:
# 12. Calculate/addBands: ChlR [Sentinel-2 only]
#
def addChlR(image):
    chlRedg = ee.Image(image).expression(
        'pow((Redg3 / Redg1), -1.0)',
        {
            'Redg1': image.select(['Redg1']),
            'Redg3': image.select(['Redg3'])
        }).float().rename('ChlR2')
    return image.addBands(chlRedg)

## PALSAR

In [ ]:
def getPALSAR(year):
    smoothing_radius = 30
    kernelType = 'circle'
    img = ee.Image("JAXA/ALOS/PALSAR/YEARLY/SAR/" + year)
    fmean = img.select('H.').focal_mean(smoothing_radius, kernelType, 'meters')
    dB = fmean.select('H.').pow(2).log10().multiply(10).add(-83.0)
    ratio = dB.select('HV').subtract(dB.select('HH')).rename('HVrHH').float()
    return dB.addBands(ratio).float()

In [ ]:
palsar20 = getPALSAR('2020')

In [ ]:
print(palsar20.getInfo())

In [ ]:
image = image.addBands(palsar20)

In [ ]:
Map.addLayer(palsar20.select(['HH','HV','HVrHH']), {'bands': ['HH','HV','HVrHH'], 'min':-16, 'max':-5, 'gamma': 0.90}, 'PALSAR 2020')

In [ ]:
Map.addLayer(img, {}, 'Image...')

In [ ]:
Map.addLayer(train_dat, {'colour': 'yellow'}, 'train data s2Coll')

In [ ]:
Map = geemap.Map(center=(-13.5, 48.4), zoom=8)
Map

In [ ]:
# del(image)

## JRC Global Surface Water

In [ ]:
# xx. Direct outputs from JRC's GSW1_3 Global Surface Water:
#
jrcImage = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")

In [ ]:
# Add the 6 standard JRC Bands to the image stack, or (see next):
#
image = image.addBands(jrcImage.unmask(0).select(['occurrence','change_abs','change_norm','seasonality','transition','max_extent'])
                     .rename(['occurrence','change_abs','change_norm','seasonality','transition','max_extent']))

In [ ]:
# xx. Retrieve JRC / addBands and rename (rename: 'change_norm' & 'seasonality' —> 'seasonalit', 10-char max.):  
#
def addJRC(image):
    # Update the mask (??)
    image = image.unmask(0)
    image = image.addBands(jrcImage.select(['occurrence']).rename(['occurrence']))
    image = image.addBands(jrcImage.select(['change_abs']).rename(['change_abs']))
    image = image.addBands(jrcImage.select(['change_norm']).rename(['change_nor']))
    image = image.addBands(jrcImage.select(['seasonality']).rename(['seasonalit']))
    image = image.addBands(jrcImage.select(['transition']).rename(['transition']))
    image = image.addBands(jrcImage.select(['max_extent']).rename(['max_extent']))
    return image #.select(['occurrence','change_abs','change_nor','seasonalit','transition','max_extent'])

In [ ]:
del(jrc)
jrc = addJRC(jrcImage)

## DEM: SRTM, NASADEM, and ALOS 'AW3D30' DSM

In [ ]:
# Three DEM versions, NASADEM should be an improvement ... need to check the band contents, as it differs between versions ***
#
srtm = ee.Image("USGS/SRTMGL1_003")
srtm1 = ee.Image("NASA/NASADEM_HGT/001")
aDem = ee.Image("JAXA/ALOS/AW3D30/V3_2").select('DSM')

In [ ]:
import math

In [ ]:
# xx. Calculate/addBands Topographic Indices: (i) slope and aspect; (ii) east- and northness
#
import math
def addTopo(image):
    topo = ee.Algorithms.Terrain(image)
  # From aspect (a), calculate 'eastness (sin a), northness (cos a)
    deg2rad = ee.Number(math.pi).divide(180)
    aspect = topo.select('aspect')
    aspect_rad = aspect.multiply(deg2rad)
    eastness = aspect_rad.sin().rename('eastness').float()
    northness = aspect_rad.cos().rename('northness').float()
    
    topo = topo.select(['slope', 'aspect']).addBands(eastness).addBands(northness)
    image = image.addBands(topo)
    return image

In [ ]:
image = image.addBands(addTopo(srtm))

In [ ]:
# Testing outpus from 'ee.Algorithms.Terrain':
#
topo = ee.Algorithms.Terrain(srtm)

## Training 

In [ ]:
dTrain_df = pd.read_csv('C:/Users/pierd/OneDrive/GIS/Madagascar/mangrove/_train/dTrain_S2sr_vJun08.csv')

In [ ]:
dTrain_df

In [ ]:
feature_names = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','NDVI','ndviSD','gNDVI','NBR','EVI','EVI2','IBI','NDWI','MNDWI','SAVI','NDBI',
'VH','VV','VHrVV','elevation','slope','aspect','eastness','northness',
'occurrence','seasonality','max_extent','change_abs','change_norm','transition','recurrence']

lc = 'lc'

In [ ]:
x = dTrain_df[feature_names]
y = dTrain_df[lc]

In [ ]:
x

In [ ]:
n_trees = 500
rf_1 = ensemble.RandomForestClassifier(n_trees).fit(x,y)

In [ ]:
rf_1

In [ ]:
trees = ml.rf_to_strings(rf_1, feature_names)

In [ ]:
# print(trees[2])
len(trees) == n_trees

In [ ]:
ee_classifier = ml.strings_to_classifier(trees)

#### Insert new image stack here:

In [ ]:
img_classified = image.select(bandSel).classify(ee_classifier) ## should match 'feature_names' with 'bandSel'

#### Saving 'trees' to ee.FeatureCollection

In [ ]:
# Specify an Asset id where to save the 'trees':
#
asset_id = "projects/ee-pierdubo/assets/rf_1v01_fc_str"

In [ ]:
ml.export_trees_to_fc(trees, asset_id)

In [ ]:
# Read the exported 'tree' feature collection
rf_1v01_fc = ee.FeatureCollection("projects/ee-pierdubo/assets/rf_1v01_fc_str")

# Convert it to a classifier, similar to the one usin function 'ml.trees_to_classifier'
rf_1v01_ = ml.fc_to_classifier(rf_1v01_fc)

In [ ]:
dTrain.size().getInfo()

In [ ]:
# dTrain2_df = pd.read_fc(dTrain)

In [ ]:
out_file

In [ ]:
geemap.extract_values_to_points(dTrain, image, out_file, scale=30, crs='EPSG:32738')

In [ ]:
bandSel = (['BLUE','GREEN','RED','Redg1','Redg2','Redg3','NIR','NIRa','SWIR1','SWIR2',
            'NDVI','ndviSD','gNDVI','NBR','EVI','EVI2','NDMI','NDWI','MNDWI','MVI','ARI2','ChlR','BSI','IBI','SAVI',
            'VV_1','VH_1','VHrVV_1','NDS1_1','VV_2','VH_2','VHrVV_2','NDS1_2','HH','HV','HVrHH',
            'change_abs','change_norm','max_extent','occurrence','seasonality','transition',
            'aspect','eastness','elevation','northness','slope'])  # 

In [ ]:
bandSel1 = (['BLUE','GREEN','RED','Redg1','Redg2','Redg3','NIR','SWIR1','SWIR2']) # NIRa',
            # 'NDVI','ndviSD','NBR','EVI','NDMI','NDWI','MNDWI','ARI2','ChlR','MVI','BSI','IBI','SAVI']) # ''gNDVI','EVI2'
            # 'VV_1','VH_1','VHrVV_1','NDS1_1','VV_2','VH_2','VHrVV_2','NDS1_2','HH','HV','HVrHH',
            # 'change_abs','change_nor','max_extent','occurrence','seasonalit','transition',
            # 'aspect','eastness','elevation','northness','slope'])

In [ ]:
bandSel2 = (['NDVI','ndviSD','NBR','EVI','NDMI','NDWI','MNDWI','ARI2','ChlR','MVI','BSI','IBI','SAVI']) # 'gNDVI','EVI2'
            # 'VV_1','VH_1','VHrVV_1','NDS1_1','VV_2','VH_2','VHrVV_2','NDS1_2','HH','HV','HVrHH',
            # 'change_abs','change_nor','max_extent','occurrence','seasonalit','transition',
            # 'aspect','eastness','elevation','northness','slope'])

In [ ]:
bandSel3 = (['VV_1','VH_1','VHrVV_1','NDS1_1','VV_2','VH_2','VHrVV_2','NDS1_2','HH','HV','HVrHH'])
            # 'change_abs','change_nor','max_extent','occurrence','seasonalit','transition',
            # 'aspect','eastness','elevation','northness','slope'])

In [ ]:
bandSel4 = (['change_abs','change_nor','max_extent','occurrence','seasonalit','transition',
            'aspect','eastness','elevation','northness','slope'])

In [ ]:
props = geemap.image_props(s2Coll.first())
props.getInfo()

In [ ]:
# Training data, random selection
#
train_rnd = image.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

In [ ]:
var train_dat = image.sample(
    collection = nfc
    properties = ['lc']
    scale: 10
    projection: 'EPSG:32649'
    tileScale: 4
    geometries: True
)

In [ ]:
train_dat = s2Coll.median().clip(geom).select(bandSel1).sampleRegions(
    **{'collection': nfc, 'properties': ['lc'], 'scale': 10, 'geometries': True}
)  # 'tileScale': 8,

In [ ]:
train_dat = image.clip(geom).select(bandSel).sampleRegions(
    **{'collection': nfc, 'properties': ['lc'], 'scale': 10, 'geometries': False}
)

In [ ]:
train_dat1 = image.clip(geom).select(bandSel1).sampleRegions(
    **{'collection': nfc, 'properties': ['lc'], 'scale': 10, 'geometries': False}
)

In [ ]:
train_dat2 = image.select(bandSel2).sampleRegions(
    **{'collection': nfc, 'properties': ['lc'], 'scale': 10, 'geometries': False}
)

In [ ]:
train_dat3 = image.select(bandSel3).sampleRegions(
    **{'collection': nfc, 'properties': ['lc'], 'scale': 10, 'geometries': False}
)

In [ ]:
train_dat4 = image.select(bandSel4).sampleRegions(
    **{'collection': nfc, 'properties': ['lc'], 'scale': 10, 'geometries': False}
)

In [ ]:
rf_1 = ee.Classifier.smileRandomForest(100).setOutputMode('CLASSIFICATION').train(train_dat, 'lc', bandSel1)

In [ ]:
dict_dict = rf_1.explain()

In [ ]:
var_imp = ee.Feature(None, ee.Dictionary(dict_dict).get('importance'))
oob_error = ee.Feature(None, ee.Dictionary(dict_dict).get('outOfBagErrorEstimate'))

In [ ]:
print('Var Imp:', var_imp.getInfo())

In [ ]:
dict_dict.get('importance').getInfo()

In [ ]:
dict_dict.get('outOfBagErrorEstimate').getInfo()

In [ ]:
img_rfClass = image.select(bandSel).classify(rf_1)

In [ ]:
task = ee.batch.Export.table.toAsset(
    collection = train_dat1,
    description = 'train_dat1',
    assetId = 'users/projects/ee-pierdubo')

In [ ]:
task.start()

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'train_dat1.csv')

In [ ]:
out_dir

In [ ]:
print(train_dat1.size().getInfo())

In [ ]:
geemap.ee_to_shp(train_dat, filename='C:/Users/pierd/Downloads/train_dat.shp')

In [ ]:
geemap.ee_to_shp(train_dat2, filename='C:/Users/pierd/Downloads/train_dat2.shp')

In [ ]:
geemap.ee_to_shp(train_dat3, filename='C:/Users/pierd/Downloads/train_dat3.shp')

In [ ]:
geemap.ee_to_shp(train_dat4, filename='C:/Users/pierd/Downloads/train_dat4.shp')

In [ ]:
# geemap.ee_export_vector_to_drive(
#     train_dat, description='train_dat', folder='0_GEE', scale=10
# )

In [ ]:
# import ee.mapclient
# ee.mapclient.centerMap(48.8450, -13.2733, 12)

In [ ]:
pprint.pprint(train_dat.limit(10).getInfo())

In [ ]:
nfc_buf = nfc.map(lambda f: f.buffer(100))

In [ ]:
nfc_union = nfc_buf.union(image)

In [ ]:
# ee.mapclient.addToMap(nfc_union, {'color': '800080'})

In [ ]:
print(palsar20.getInfo())

In [ ]:
pp = pprint.PrettyPrinter(depth=6)

In [ ]:
pp.pprint(palsar20.getInfo())

In [ ]:
geom_1 = Map.user_roi

In [ ]:
p = palsar20.sample(geom_1, 100).first().get('HH').getInfo()

In [ ]:
print(p)

In [ ]:
srtm = ee.Image("USGS/SRTMGL1_003")
elev = srtm.select('elevation')

In [ ]:
elv = srtm.getRegion(geom_1, 1000).getInfo()

In [ ]:
s1_ = s1Coll.getRegion(geom_1, 1000).getInfo()

In [ ]:
s1_[:5]

In [ ]:
s2sub = s2Coll.select('SWIR1').getRegion(geom_1, 300).getInfo()

In [ ]:
s2sub[:6]

### ee to DataFrame

In [ ]:
import pandas as pd

In [ ]:
def ee_array_to_df(arr, list_of_bands):
    df = pd.DataFrame(arr)
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')
        df['datetime'] = pd.to_datetime(df['time'], unit = 'ms')
        df = df[['time', 'datetime', *list_of_bands]]
        
        return df

In [ ]:
vImp_df = ee_array_to_df(vImp, vImp(1))

In [ ]:
train_dat_df = ee_array_to_df(train_dat, ['SWIR1'])

In [ ]:
s1VH_df = ee_array_to_df(s1_, ['VH']) 

In [ ]:
s1VH_df.head()

In [ ]:
s2SWIR1_df = ee_array_to_df(s2sub, ['SWIR1'])

In [ ]:
s2SWIR1_df.head()

## Charts

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
%matplotlib inline

In [ ]:
vImp = dict_dict.get('importance').getInfo()

In [ ]:
print(dict_dict.limit(10).getInfo())

In [ ]:
plt.bar(var_imp)
plt.show()

In [ ]:
x_data_swir1 = np.asanyarray(s2SWIR1_df['time'].apply(float))

In [ ]:
y_data_swir1 = np.asanyarray(s2SWIR1_df['SWIR1'].apply(float))

In [ ]:
def fit_fun(x, a, b, c):
    return a*(np.sin(b*x + c) + 1)

In [ ]:
params_sw1, params_covariance_sw1 = optimize.curve_fit(
    fit_fun, x_data_swir1, y_data_swir1, p0 = [20., 0.002*np.pi/(365.*24.*3600.), 350])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.hist(s2SWIR1_df['SWIR1'], bins=200, range=(0, 0.2))
# ax.scatter(s2SWIR1_df['datetime'], s2SWIR1_df['SWIR1'])
# ax.plot(s2SWIR1_df['datetime'],
#         fit_fun(x_data_swir1, params_sw1[0], params_sw1[1], params_sw1[2]),
#         label='VH fitted', color='red', lw=1.5)
ax.set_xlabel('Date', fontsize=10)
ax.set_ylabel('SWIR1 SR', fontsize=12)
ax.grid(lw=0.2)
# ax.legend(fontsize=10, loc='lower right')

plt.show()

In [ ]:
x_data_vh = np.asanyarray(s1VH_df['time'].apply(float))

In [ ]:
y_data_vh = np.asanyarray(s1VH_df['VH'].apply(float))

In [ ]:
vh0 = -14
delta_vh = 10
tau = 365*24*3600*1000
phi = 2*np.pi*4*30.5*3600*1000/tau

In [ ]:
def fit_func(t, vh0, delta_vh, tau, phi):
    return vh0 + (delta_vh/2)*np.sin(2*np.pi*t/tau + phi)

In [ ]:
params_vh, params_covariance_vh = optimize.curve_fit(
    fit_func, x_data_vh, y_data_vh, p0=[vh0, delta_vh, tau, phi])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter(s1VH_df['datetime'], s1VH_df['VH'])
ax.plot(s1VH_df['datetime'],
        fit_func(x_data_vh, params_vh[0], params_vh[1], params_vh[2], params_vh[3]),
        label='VH fitted', color='red', lw=1.5)
ax.set_xlabel('Date', fontsize=10)
ax.set_ylabel('VH (dB)', fontsize=12)
ax.grid(lw=0.2)
ax.legend(fontsize=10, loc='lower right')
plt.show()

In [ ]:
# No such thing/use as 'ui' in the python API
#
S1Chart1 = (ui.Chart.image.series(s1Coll.select('VHrVV'), geom)
            .setChartType('ScatterChart')
            .setOptions({
                title: 'S1 timeseries VH/VV',
                trendlines: {0: {color: 'CC0000'}},
                lineWidth: 1,
                pointSize: 3,
            }))                